In [ ]:
import os
import cv2
from utils import createDirectory

mp4 to jpg

In [ ]:

file_path = '/opt/ml/input/data/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A.mp4'
createDirectory(file_path[:-4])

start = file_path.find('/', file_path.find('/', file_path.find('/', file_path.find('/') + 1)+1)+1)

video_name = file_path[start+1:-4]

cap = cv2.VideoCapture(file_path)
FPS = cap.get(5) 
frame = cap.get(7)
count = 1

if FPS == 30:
    for i in range(1, int(frame)):
        success, img = cap.read()
        
        if not success:
            break

        if i % (1.25) < 1:
            cv2.imwrite(file_path[:-4] + f"/{video_name}"+'-'+str(count).zfill(6)+'.jpg', img)
            count+=1
elif FPS == 24:
    # for i in range(1, int(frame)):
    for i in range(1, 50):
        success, img = cap.read()
        
        if not success:
            break

        cv2.imwrite(file_path[:-4] + f"/{video_name}"+'-'+str(count).zfill(6)+'.jpg', img)
        count+=1


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable

import torchvision
from torchvision import datasets, transforms

import videotransforms

import numpy as np
from pytorch_i3d import InceptionI3d
from charades_dataset_full import Charades as Dataset

In [2]:
test_transforms = transforms.Compose([])
first_transforms = transforms.Compose([videotransforms.FirstCrop(224)])
second_transforms = transforms.Compose([videotransforms.SecondCrop(224)])
third_transforms = transforms.Compose([videotransforms.ThirdCrop(224)])
fourth_transforms = transforms.Compose([videotransforms.FourthCrop(224)])
center_transforms = transforms.Compose([videotransforms.CenterCrop(224)])

In [3]:
split = '/opt/ml/input/data/test.json'
root = '/opt/ml/input/data'
mode = 'rgb'
batch_size = 1
save_dir = '/opt/ml/input/data'
load_model = '/opt/ml/input/code/project/pytorch-i3d/models/rgb_imagenet.pt'

first_dataset = Dataset(split, 'testing', root, mode, first_transforms, num=-1, save_dir=save_dir)
first_dataloader = torch.utils.data.DataLoader(first_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

second_dataset = Dataset(split, 'testing', root, mode, second_transforms, num=-1, save_dir=save_dir)
second_dataloader = torch.utils.data.DataLoader(second_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

third_dataset = Dataset(split, 'testing', root, mode, third_transforms, num=-1, save_dir=save_dir)
third_dataloader = torch.utils.data.DataLoader(third_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

fourth_dataset = Dataset(split, 'testing', root, mode, fourth_transforms, num=-1, save_dir=save_dir)
fourth_dataloader = torch.utils.data.DataLoader(fourth_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

center_dataset = Dataset(split, 'testing', root, mode, center_transforms, num=-1, save_dir=save_dir)
center_dataloader = torch.utils.data.DataLoader(center_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)


dataloaders = {
               '0': first_dataloader,
               '1': second_dataloader,
               '2': third_dataloader,
               '3': fourth_dataloader,
               '4': center_dataloader,
                }

datasets = {
            '0': first_dataset,
            '1': second_dataset,
            '2': third_dataset,
            '3': fourth_dataset,
            '4': center_dataset,
            }  

i3d = InceptionI3d(400, in_channels=3)
i3d.replace_logits(400)
i3d.load_state_dict(torch.load(load_model))
i3d.cuda()
i3d.train(False)  # Set model to evaluate mode
        
# Iterate over data.
for phase in ['0', '1', '2', '3', '4']:
    for data in dataloaders[phase]:
        # get the inputs
        inputs, labels, name = data
        b,c,t,h,w = inputs.shape
        features = []
        for start in range(1, t, 16):
            end = start + 16 
            ip = Variable(torch.from_numpy(inputs.numpy()[:,:,start:end]).cuda())
            features.append(i3d.extract_features(ip).squeeze(0).permute(1,2,3,0).data.cpu().numpy())
        # np.save(os.path.join(save_dir, name[0]+f"_{phase}"), np.concatenate(features, axis=0))
        np.save(os.path.join(save_dir, name[0]+f"__{phase}"), np.concatenate(features, axis=0).reshape(-1, 1024))
        print(f"Success Saved : {os.path.join(save_dir, name[0]+'__'+phase)}.npy")    

Success Saved : /opt/ml/input/data/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A__0.npy
Success Saved : /opt/ml/input/data/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A__1.npy
Success Saved : /opt/ml/input/data/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A__2.npy
Success Saved : /opt/ml/input/data/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A__3.npy
Success Saved : /opt/ml/input/data/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A__4.npy


In [ ]:
first_data = np.load('/opt/ml/input/data/test_video_1.npy')
second_data = np.load('/opt/ml/input/data/test_video_2.npy')
third_data = np.load('/opt/ml/input/data/test_video_3.npy')
fourth_data = np.load('/opt/ml/input/data/test_video_4.npy')
center_data = np.load('/opt/ml/input/data/test_video_5.npy')

print('first_crop :', first_data.shape)
print('second_crop :', second_data.shape)
print('third_crop :', third_data.shape)
print('fourth_crop :', fourth_data.shape)
print('center_crop :', center_data.shape)

In [7]:
first_data = np.load('/opt/ml/input/data/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A__0.npy')
second_data = np.load('/opt/ml/input/data/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A__1.npy')
third_data = np.load('/opt/ml/input/data/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A__2.npy')
fourth_data = np.load('/opt/ml/input/data/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A__3.npy')
center_data = np.load('/opt/ml/input/data/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A__4.npy')

print('first_crop :', first_data.shape)
print('second_crop :', second_data.shape)
print('third_crop :', third_data.shape)
print('fourth_crop :', fourth_data.shape)
print('center_crop :', center_data.shape)

first_crop : (360, 1024)
second_crop : (360, 1024)
third_crop : (360, 1024)
fourth_crop : (360, 1024)
center_crop : (360, 1024)


In [8]:
first_data_gt = np.load('/opt/ml/input/data/gt/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A__0.npy')
second_data_gt = np.load('/opt/ml/input/data/gt/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A__1.npy')
third_data_gt = np.load('/opt/ml/input/data/gt/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A__2.npy')
fourth_data_gt = np.load('/opt/ml/input/data/gt/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A__3.npy')
center_data_gt = np.load('/opt/ml/input/data/gt/A.Beautiful.Mind.2001__#00-25-20_00-29-20_label_A__4.npy')

print('first_crop :', first_data_gt.shape)
print('second_crop :', second_data_gt.shape)
print('third_crop :', third_data_gt.shape)
print('fourth_crop :', fourth_data_gt.shape)
print('center_crop :', center_data_gt.shape)

first_crop : (360, 1024)
second_crop : (360, 1024)
third_crop : (360, 1024)
fourth_crop : (360, 1024)
center_crop : (360, 1024)


In [9]:
first_data[0][:10], first_data_gt[0][:10]

(array([0.19726132, 0.05483772, 0.15460849, 0.08884379, 0.2266495 ,
        0.11017099, 0.15304732, 0.22752951, 0.13596202, 0.17326353],
       dtype=float32),
 array([0.39029422, 0.1425842 , 0.37637228, 0.25315678, 0.24167438,
        0.14017773, 0.1671794 , 0.177893  , 0.18694301, 0.10136556],
       dtype=float32))